In [1]:
from huggingface_hub import login
from secret import HF_TOKEN

login(
  token=HF_TOKEN, 
  add_to_git_credential=True
)

In [2]:
import torch
print("CUDA available:", torch.cuda.is_available())

CUDA available: True


In [3]:
from datasets import load_dataset

ds = load_dataset("Glebosol/SPB_sm_pablic")

print(len(ds['train']))

2934


In [4]:
ds['train']

Dataset({
    features: ['Unnamed: 0', 'text', 'label'],
    num_rows: 2934
})

In [5]:
from random import randrange

random_id = randrange(len(ds['train']))
ds['train'][random_id]

{'Unnamed: 0': 450,
 'text': 'здравствуйте может подойти отец детей с моим паспортом формой  детей и их сввыми о рождение все кроме мужа прописаны на острове декабристов у меня нет возможности самой подойти',
 'label': 0}

In [6]:
from transformers import AutoTokenizer


model_id = "cointegrated/rubert-tiny2"
tokenizer = AutoTokenizer.from_pretrained(model_id)

def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, return_tensors="pt")

raw_dataset =  ds.rename_column("label", "labels") 
tokenized_dataset = raw_dataset.map(tokenize, batched=True,remove_columns=["text"])
tokenized_dataset = tokenized_dataset["train"].train_test_split(test_size=0.2)

print(tokenized_dataset["train"].features.keys())

dict_keys(['Unnamed: 0', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'])


In [7]:
from transformers import AutoModelForSequenceClassification


model_id = "cointegrated/rubert-tiny2"

num_labels = len(set(tokenized_dataset["train"]["labels"]))  


model = AutoModelForSequenceClassification.from_pretrained(
    model_id, num_labels=num_labels
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
import evaluate
import numpy as np


metric = evaluate.load("f1")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

In [9]:
from huggingface_hub import HfFolder
from transformers import Trainer, TrainingArguments
import torch._dynamo
torch._dynamo.config.suppress_errors = True

# Id for remote repository
repository_id = "Glebosol/sm_problem_cat"

# Define training args
training_args = TrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    learning_rate=5e-5,
	num_train_epochs=3,
	# PyTorch 2.0 specifics
    bf16=True, # bfloat16 training
	torch_compile=True, # optimizations
    optim="adamw_torch_fused", # improved optimizer
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=200,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    # push to hub parameters
    report_to="tensorboard",
    push_to_hub=True,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),

)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
trainer.train()

W0121 23:41:17.912000 21640 site-packages\torch\_dynamo\convert_frame.py:1125] WON'T CONVERT forward c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\bert\modeling_bert.py line 1636 
W0121 23:41:17.912000 21640 site-packages\torch\_dynamo\convert_frame.py:1125] due to: 
W0121 23:41:17.912000 21640 site-packages\torch\_dynamo\convert_frame.py:1125] Traceback (most recent call last):
W0121 23:41:17.912000 21640 site-packages\torch\_dynamo\convert_frame.py:1125]   File "c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_dynamo\output_graph.py", line 1446, in _call_user_compiler
W0121 23:41:17.912000 21640 site-packages\torch\_dynamo\convert_frame.py:1125]     compiled_fn = compiler_fn(gm, self.example_inputs())
W0121 23:41:17.912000 21640 site-packages\torch\_dynamo\convert_frame.py:1125]                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
W0121 23:41:17.912000 21640 site-packages\torch\_dynamo\convert_frame.py:11

Epoch,Training Loss,Validation Loss


W0122 00:17:47.209000 21640 site-packages\torch\_dynamo\convert_frame.py:1125] WON'T CONVERT forward c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\accelerate\utils\operations.py line 818 
W0122 00:17:47.209000 21640 site-packages\torch\_dynamo\convert_frame.py:1125] due to: 
W0122 00:17:47.209000 21640 site-packages\torch\_dynamo\convert_frame.py:1125] Traceback (most recent call last):
W0122 00:17:47.209000 21640 site-packages\torch\_dynamo\convert_frame.py:1125]   File "c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_dynamo\output_graph.py", line 1446, in _call_user_compiler
W0122 00:17:47.209000 21640 site-packages\torch\_dynamo\convert_frame.py:1125]     compiled_fn = compiler_fn(gm, self.example_inputs())
W0122 00:17:47.209000 21640 site-packages\torch\_dynamo\convert_frame.py:1125]                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
W0122 00:17:47.209000 21640 site-packages\torch\_dynamo\convert_frame.py:1125]   File "

KeyboardInterrupt: 

In [ ]:
tokenizer.save_pretrained(repository_id)
trainer.create_model_card()
trainer.push_to_hub()